# Wichtig

Importieren der Module

In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
from numpy.linalg import norm
from dtaidistance import dtw
from fastdtw import fastdtw
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import plotly.graph_objs as go

einlesen der einzelnen Tabelle in unterschiedliche Dataframes

In [20]:
# CSV-Datei einlesen
df = pd.read_csv("werte_daten/werte_land_PM2_5.csv")

# Datumsspalte in DateTime-Objekte konvertieren
df['createdAt'] = pd.to_datetime(df['createdAt'])

# Mittelwert der Spalten 'value_1', 'value_2' und 'value_3' berechnen
df['land_PM2_5'] = df[['value_1', 'value_2', 'value_3', 'value_4', 'value_5']].mean(axis=1)

# DataFrame auf ausgewählte Spalten reduzieren und createdAt als Index setzen
selected_df1 = df[['createdAt', 'land_PM2_5']].set_index('createdAt')
# Duplikate im Index entfernen
selected_df1 = selected_df1[~selected_df1.index.duplicated(keep='first')]
# Remove rows with NaN values
selected_df1 = selected_df1.dropna()

In [21]:
# CSV-Datei einlesen
df = pd.read_csv("werte_daten/werte_land_PM10.csv")

# Datumsspalte in DateTime-Objekte konvertieren
df['createdAt'] = pd.to_datetime(df['createdAt'])

# Mittelwert der Spalten 'value_1', 'value_2' und 'value_3' berechnen
df['land_PM10'] = df[['value_1', 'value_2', 'value_3', 'value_4', 'value_5']].mean(axis=1)

# DataFrame auf ausgewählte Spalten reduzieren und createdAt als Index setzen
selected_df2 = df[['createdAt', 'land_PM10']].set_index('createdAt')
# Duplikate im Index entfernen
selected_df2 = selected_df2[~selected_df2.index.duplicated(keep='first')]
# Remove rows with NaN values
selected_df2 = selected_df2.dropna()

In [22]:
# CSV-Datei einlesen
df = pd.read_csv("werte_daten/werte_stadt_PM2_5.csv")

# Datumsspalte in DateTime-Objekte konvertieren
df['createdAt'] = pd.to_datetime(df['createdAt'])

# Mittelwert der Spalten 'value_1', 'value_2' und 'value_3' berechnen
df['stadt_PM2_5'] = df[['value_1', 'value_2', 'value_3', 'value_4', 'value_5']].mean(axis=1)

# DataFrame auf ausgewählte Spalten reduzieren und createdAt als Index setzen
selected_df3 = df[['createdAt', 'stadt_PM2_5']].set_index('createdAt')
# Duplikate im Index entfernen
selected_df3 = selected_df3[~selected_df3.index.duplicated(keep='first')]
# Remove rows with NaN values
selected_df3 = selected_df3.dropna()

In [23]:
# CSV-Datei einlesen
df = pd.read_csv("werte_daten/werte_stadt_PM10.csv")

# Datumsspalte in DateTime-Objekte konvertieren
df['createdAt'] = pd.to_datetime(df['createdAt'])

# Mittelwert der Spalten 'value_1', 'value_2' und 'value_3' berechnen
df['stadt_PM10'] = df[['value_1', 'value_2', 'value_3', 'value_4', 'value_5']].mean(axis=1)

# DataFrame auf ausgewählte Spalten reduzieren und createdAt als Index setzen
selected_df4 = df[['createdAt', 'stadt_PM10']].set_index('createdAt')
# Duplikate im Index entfernen
selected_df4 = selected_df4[~selected_df4.index.duplicated(keep='first')]
# Remove rows with NaN values
selected_df4 = selected_df4.dropna()

# nicht mehr wichtig

die Dataframes werden verbunden

In [24]:
# DataFrames entlang der Spalten verbinden
df_combined = pd.concat([selected_df1, selected_df2, selected_df3, selected_df4], axis=1)

nun wird für die einzelnen Monate untersucht, welche Varianz und Standardabweichung die Werte haben und graphisch in einem Boxplot dargestellt

In [27]:
df_reset = df_combined.reset_index()

# Konvertieren der 'createdAT'-Spalte in Datetime und Extrahieren des Monats
df_reset['createdAt'] = pd.to_datetime(df_reset['createdAt'], format='%Y/%m')
df_reset['Monat'] = df_reset['createdAt'].dt.month

# Gruppieren nach Monat und Berechnen der Varianz und Standardabweichung
monatliche_statistiken = df_reset.groupby('Monat').agg({
    'land_PM2_5': ['var', 'std'],
    'land_PM10': ['var', 'std'],
    'stadt_PM2_5': ['var', 'std'],
    'stadt_PM10' : ['var', 'std']
})

# Daten umstrukturieren für Plotly Express
melted_df = pd.melt(df_reset, id_vars=['Monat'], value_vars=['land_PM2_5', 'land_PM10', 'stadt_PM2_5', 'stadt_PM10'],
                    var_name='Temperaturtyp', value_name='Temperatur')

# Boxplot erstellen
fig = px.box(melted_df, x='Monat', y='Temperatur', color='Temperaturtyp', title='Boxplots der Temperaturwerte (1980-2023)')
fig.update_layout(xaxis_title='Monat', yaxis_title='Temperatur (°C)', boxmode='overlay')
fig.show()
print(monatliche_statistiken)

      land_PM2_5             land_PM10            stadt_PM2_5            \
             var       std         var        std         var       std   
Monat                                                                     
1      26.663648  5.163686  116.725575  10.803961   25.185163  5.018482   
2      21.742775  4.662915   42.885786   6.548724   47.839660  6.916622   
3      34.628925  5.884635   58.623351   7.656589   72.700485  8.526458   
4       6.679191  2.584413   16.749846   4.092658   10.361329  3.218902   
5       3.283914  1.812157    5.306801   2.303649    4.699004  2.167719   
6       2.882810  1.697884    4.522827   2.126694    4.618927  2.149169   
7       1.698404  1.303228    3.462548   1.860792    4.987687  2.233313   
8       4.980364  2.231673    9.827771   3.134928    7.987185  2.826161   
9       6.455336  2.540735    8.151579   2.855097    3.715820  1.927646   
10      2.607267  1.614703    7.306046   2.702970    6.490124  2.547572   
11     16.497481  4.06170

es werden nun die unterschiedlichen Zeitreihen nach Trend, Saisonalität und Residuen unterteilt und dargestellt

es wird für die Werte PM2.5 auf dem Land erledigt

In [28]:
# Datumspalte in Datetime-Format umwandeln und als Index setzen
selected_df1.index = pd.to_datetime(selected_df1.index, format='%Y/%m')
# die Frequenz der Werte wird auf täglich gesetzt
selected_df1 = selected_df1.asfreq('D')
# Fehlende Daten interpolieren
selected_df1['land_PM2_5'] = selected_df1['land_PM2_5'].interpolate(method='linear')

In [29]:
# Dekomposition der Zeitreihe
result = seasonal_decompose(selected_df1['land_PM2_5'], model='additive')

# Trend, Saisonalität extrahieren
trend = result.trend
seasonal = result.seasonal
residual = result.resid

# Ergebnisse in das DataFrame einfügen
selected_df1['Trend'] = trend
selected_df1['Saisonalität'] = seasonal
selected_df1['Residuen'] = residual

# Daten für Plotly Express vorbereiten
df_plotly = selected_df1.reset_index()

# Plot mit Plotly Express erstellen
fig = px.line(df_plotly, x='createdAt', y=['land_PM2_5', 'Trend', 'Saisonalität', 'Residuen'],
              labels={'value': 'land_PM2_5', 'variable': 'Komponente', 'Datum': 'Datum'},
              title='Dekomposition der monatlichen Werte PM2.5 auf dem Land')
fig.show()

es wird für die Werte PM10 auf dem Land erledigt

In [30]:
# Datumspalte in Datetime-Format umwandeln und als Index setzen
selected_df2.index = pd.to_datetime(selected_df2.index, format='%Y/%m')
selected_df2 = selected_df2.asfreq('D')
# Fehlende Daten interpolieren
selected_df2['land_PM10'] = selected_df2['land_PM10'].interpolate(method='linear')

# Dekomposition der Zeitreihe
result = seasonal_decompose(selected_df2['land_PM10'], model='additive')

# Trend, Saisonalität extrahieren
trend = result.trend
seasonal = result.seasonal
residual = result.resid

# Ergebnisse in das DataFrame einfügen
selected_df2['Trend'] = trend
selected_df2['Saisonalität'] = seasonal
selected_df2['Residuen'] = residual

# Daten für Plotly Express vorbereiten
df_plotly = selected_df2.reset_index()

# Plot mit Plotly Express erstellen
fig = px.line(df_plotly, x='createdAt', y=['land_PM10', 'Trend', 'Saisonalität', 'Residuen'],
              labels={'value': 'land_PM10', 'variable': 'Komponente', 'Datum': 'Datum'},
              title='Dekomposition der monatlichen Werte PM10 auf dem Land')
fig.show()

es wird für die Werte PM2.5 in der Stadt erledigt

In [31]:
# Datumspalte in Datetime-Format umwandeln und als Index setzen
selected_df3.index = pd.to_datetime(selected_df3.index, format='%Y/%m')
selected_df3 = selected_df3.asfreq('D')
# Fehlende Daten interpolieren
selected_df3['stadt_PM2_5'] = selected_df3['stadt_PM2_5'].interpolate(method='linear')

# Dekomposition der Zeitreihe
result = seasonal_decompose(selected_df3['stadt_PM2_5'], model='additive')

# Trend, Saisonalität extrahieren
trend = result.trend
seasonal = result.seasonal
residual = result.resid

# Ergebnisse in das DataFrame einfügen
selected_df3['Trend'] = trend
selected_df3['Saisonalität'] = seasonal
selected_df3['Residuen'] = residual

# Daten für Plotly Express vorbereiten
df_plotly = selected_df3.reset_index()

# Plot mit Plotly Express erstellen
fig = px.line(df_plotly, x='createdAt', y=['stadt_PM2_5', 'Trend', 'Saisonalität', 'Residuen'],
              labels={'value': 'stadt_PM2_5', 'variable': 'Komponente', 'Datum': 'Datum'},
              title='Dekomposition der monatlichen Werte PM2.5 in der Stadt')
fig.show()

es wird für die Werte PM10 auf dem Land erledigt

In [32]:
# Datumspalte in Datetime-Format umwandeln und als Index setzen
selected_df4.index = pd.to_datetime(selected_df4.index, format='%Y/%m')
selected_df4 = selected_df4.asfreq('D')
# Fehlende Daten interpolieren
selected_df4['stadt_PM10'] = selected_df4['stadt_PM10'].interpolate(method='linear')

# Dekomposition der Zeitreihe
result = seasonal_decompose(selected_df4['stadt_PM10'], model='additive')

# Trend, Saisonalität extrahieren
trend = result.trend
seasonal = result.seasonal
residual = result.resid

# Ergebnisse in das DataFrame einfügen
selected_df4['Trend'] = trend
selected_df4['Saisonalität'] = seasonal
selected_df4['Residuen'] = residual

# Daten für Plotly Express vorbereiten
df_plotly = selected_df4.reset_index()

# Plot mit Plotly Express erstellen
fig = px.line(df_plotly, x='createdAt', y=['stadt_PM10', 'Trend', 'Saisonalität', 'Residuen'],
              labels={'value': 'stadt_PM10', 'variable': 'Komponente', 'Datum': 'Datum'},
              title='Dekomposition der monatlichen Werte PM10 in der Stadt')
fig.show()

nun wird die Autokorrelation der einzelnen Werte für verschiedene Lags betrachtet

es wird für die Werte PM2.5 auf dem Land erledigt

In [33]:
# Autokorrelationen für spezifische Lags berechnen
lags = [1, 3, 6, 9, 12]
acf_values = sm.tsa.acf(selected_df1["land_PM2_5"], nlags=max(lags))

# Nur die spezifischen Lags extrahieren
acf_specific_lags = {lag: acf_values[lag] for lag in lags}
acf_specific_lags

{1: 0.8676561041054632,
 3: 0.49866830925729433,
 6: 0.3455193913807173,
 9: 0.29663842597246565,
 12: 0.14001480474291744}

es wird für die Werte PM10 auf dem Land erledigt

In [34]:
# Autokorrelationen für spezifische Lags berechnen
lags = [1, 3, 6, 9, 12]
acf_values = sm.tsa.acf(selected_df2["land_PM10"], nlags=max(lags))

# Nur die spezifischen Lags extrahieren
acf_specific_lags = {lag: acf_values[lag] for lag in lags}
acf_specific_lags

{1: 0.8656188585440419,
 3: 0.4966116824173901,
 6: 0.32818717143709153,
 9: 0.29429686684219136,
 12: 0.1562921306509525}

es wird für die Werte PM2.5 in der Stadt erledigt

In [35]:
# Autokorrelationen für spezifische Lags berechnen
lags = [1, 3, 6, 9, 12]
acf_values = sm.tsa.acf(selected_df3["stadt_PM2_5"], nlags=max(lags))

# Nur die spezifischen Lags extrahieren
acf_specific_lags = {lag: acf_values[lag] for lag in lags}
acf_specific_lags

{1: 0.8976011309565914,
 3: 0.6081418818594634,
 6: 0.5312161009296948,
 9: 0.4178156563104783,
 12: 0.29928851815594115}

es wird für die Werte PM10 in der Stadt erledigt

In [36]:
# Autokorrelationen für spezifische Lags berechnen
lags = [1, 3, 6, 9, 12]
acf_values = sm.tsa.acf(selected_df4["stadt_PM10"], nlags=max(lags))

# Nur die spezifischen Lags extrahieren
acf_specific_lags = {lag: acf_values[lag] for lag in lags}
acf_specific_lags

{1: 0.798535350676084,
 3: 0.34598035137548794,
 6: 0.38154108678765114,
 9: 0.2662461009125047,
 12: 0.23390176322119954}

# wichtig

nun werden für die einzelnen Tabellen verschieden Methoden der Vorhersage angewandt

Zuerst kommt die Definition der angewandten Methoden, dies wird sowohl für die Traingsdaten, also den Anfang des Jahres 2024, bei dem diese dann mit den tatsächlichen Werten verglichen werden und mit dem Rest des Jahres, bei dem es noch keine Werte gibt

In [38]:
# 1. Arithmetisches Mittel
def arithmetisches_mittel(train, test):
    mean_value = train.mean()
    predictions['Arithmetisches Mittel'] = mean_value
    forecasts['Arithmetisches Mittel'] = mean_value
    return np.full_like(test, fill_value=mean_value)

# 2. Naive Methode
def naive_methode(train, test):
    last_value = train.iloc[-1]
    predictions['Naive Methode'] = last_value
    last_value = test.iloc[-1]
    forecasts['Naive Mathode'] = last_value
    return np.full_like(test, fill_value=last_value)

# 3. Saisonale naive Vorhersage (Vorhersage basierend auf dem letzten bekannten Wert derselben Saison)
def saisonale_naive(train, test):
    # Die letzten bekannten Werte jeder Saison (Monat) aus dem Trainingsdatensatz erhalten
    seasonal_last_values = train.groupby(level='Monat').apply(lambda x: x.tail(1)).values
    predictions['Saisonale naive'] = np.tile(seasonal_last_values, n_test // 12 + 1)[:n_test]
    n_forecast = len(forecasts.index)
    seasonal_last_values_test = test.groupby(level='Monat').apply(lambda x: x.tail(1)).values
    # Für die Vorhersagen der Zukunft, beginnt es im Juni
    start_month = 5
    # Berechne den Index des Startmonats in der Liste
    start_index = (start_month - 1) % 12
    # Verschiebe die Liste so, dass sie mit dem Startmonat beginnt
    shifted_values = np.concatenate((seasonal_last_values_test[start_index:], seasonal_last_values_test[:start_index]))
    forecasts['Saisonale naive'] = np.tile(shifted_values, n_forecast // 12 + 1)[:n_forecast]
    return predictions['Saisonale naive'].values

def exponentielle_glättung(train, test):
    prediction = pd.Series(index=test.index)
    prediction.iloc[0] = train.iloc[-1]  # Initialisierung des ersten Werts
    # den Alpha Wert festlegen
    alpha = 0.5
    for i in range(1, len(test)):
        prediction.iloc[i] = alpha*test.iloc[i - 1] + alpha*(1 - alpha)**2 * test.iloc[i - 2] + alpha*(1 - alpha)**3 * test.iloc[i - 3]
    predictions["Exponentielle Glättung"] = prediction
    forecast = pd.Series(index=forecasts.index)
    # hier gibt es andere Alphawerte, da es in meinen Augen nur Sinn macht, wenn es insgesamt 1 ist
    # Da die Werte sonst immer kleiner werden
    alpha1 = 0.58
    alpha2 = 0.29
    alpha3 = 0.13
    # es wird unterschieden, ob man noch den Index von den Testdaten benutzen muss oder nur die Forecast-Daten
    for i in range(0, len(forecast.index)):
        if i == 0:
            forecast.iloc[i] = alpha1*test.iloc[-1] + alpha2* test.iloc[-2] + alpha3* test.iloc[-3]
        elif i == 1:
            forecast.iloc[i] = alpha1*forecast.iloc[i - 1] + alpha2* test.iloc[-1] + alpha3* test.iloc[-2]
        elif i == 2:
            forecast.iloc[i] = alpha1*forecast.iloc[i - 1] + alpha2* forecast.iloc[i - 2] + alpha3* test.iloc[-1]
        else:
            forecast.iloc[i] = alpha1*forecast.iloc[i - 1] + alpha2* forecast.iloc[i - 2] + alpha3* forecast.iloc[i - 3]
    forecasts["Exponentielle Glättung"] = forecast
    return predictions['Exponentielle Glättung'].values

nun werden die Funktionen auf die tabellen angewandt und graphisch dargestellt

Jetzt wird die Prognose für die Tabelle der PM2.5 Werte auf dem Land gemacht

In [39]:
# Erstellen von Spalten für Jahr und Monat
selected_df1['Jahr'] = selected_df1.index.year
selected_df1['Monat'] = selected_df1.index.month

# Gruppierung nach Jahr und Monat, Berechnung des Durchschnitts
monthly_mean = selected_df1.groupby(['Jahr', 'Monat']).mean()

# Aufteilung in Trainings- und Testdaten
train = monthly_mean[monthly_mean.index < (2023, 6)]['land_PM2_5']
test = monthly_mean[monthly_mean.index >= (2023, 6)]['land_PM2_5']

# Anzahl der Testdatenpunkte
n_test = len(test)

# Platzhalter für Vorhersagen
predictions = pd.DataFrame(index=test.index)

# Erstellen des Index mit dem gewünschten Datumbereich für die Vorhersage
forecast_index = pd.period_range(start='2024-06', end='2025-06', freq='M')

# Erstellen des leeren DataFrames mit dem Index
forecasts = pd.DataFrame(index=forecast_index)

# die Funktionen werden nun ausgeführt und die Dataframes mit den berechneten Werten gefüllt
arith_predictions = arithmetisches_mittel(train, test)
naive_predictions = naive_methode(train, test)
seasonal_naive_predictions = saisonale_naive(train, test)
exp_smooth_predictions = exponentielle_glättung(train, test)

# Reset index für Predictions DataFrame
predictions_reset = predictions.reset_index()

# Schmelzen des DataFrames für Plotly Express
predictions_melted = predictions_reset.melt(id_vars=['Jahr', 'Monat'], 
                                            value_vars=['Arithmetisches Mittel', 'Naive Methode', 'Saisonale naive', 'Exponentielle Glättung'],
                                            var_name='Methode', 
                                            value_name='land_PM2_5')


# das Datum wird in das richtige Format gebracht, für das bessere darstellen beider Dataframes
predictions_melted['Datum'] = pd.to_datetime(predictions_melted['Jahr'].astype(str) + '-' + predictions_melted['Monat'].astype(str))

# Liniengrafik erstellen
fig = px.line(predictions_melted, x='Datum', y='land_PM2_5', color='Methode',
              labels={'land_PM2_5': 'land_PM2_5', 'Methode': 'Methode', 'Datum': 'Datum'},
              title='Vergleich der Vorhersagemethoden für Werte PM2.5 auf dem Land')

# Das Datum wird nun auch in das richtige Format gebracht
test.index = test.index.map(lambda x: f'{x[0]}-{x[1]:02d}')

# Hinzufügen der Testdaten
fig.add_scatter(x=test.index, y=test.values, mode='lines', name='Testdaten (2023)')

# nun wird auch der Index des Forecasts zu dem gleichen Format
forecasts.index = forecasts.index.to_timestamp()
# Linien für den Forecast erstellen
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Arithmetisches Mittel'], mode='lines', name='Arithmetisches Mittel Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Naive Mathode'], mode='lines', name='Naive Methode Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Saisonale naive'], mode='lines', name='Saisonale naive Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Exponentielle Glättung'], mode='lines', name='Exponentielle Glättung Forecast'))


# Grafik anzeigen
fig.show()

nun werden die Mean-Absolute-Error Fehler der einzelnen Methoden für die Testdaten und Predictions angegeben

In [40]:
mae_arith = mean_absolute_error(test, arith_predictions)
print(mae_arith)
mae_naive = mean_absolute_error(test, naive_predictions)
print(mae_naive)
mae_seasonal_naive = mean_absolute_error(test, seasonal_naive_predictions)
print(mae_seasonal_naive)
mae_exp_smooth = mean_absolute_error(test, exp_smooth_predictions)
print(mae_exp_smooth)

1.5695440238250786
1.5857475492316564
2.677683317295525
1.2318691204283052


In [41]:
# Erstellen von Spalten für Jahr und Monat
selected_df2['Jahr'] = selected_df2.index.year
selected_df2['Monat'] = selected_df2.index.month

# Gruppierung nach Jahr und Monat, Berechnung des Durchschnitts
monthly_mean = selected_df2.groupby(['Jahr', 'Monat']).mean()

# Aufteilung in Trainings- und Testdaten
train = monthly_mean[monthly_mean.index < (2023, 6)]['land_PM10']
test = monthly_mean[monthly_mean.index >= (2023, 6)]['land_PM10']

# Anzahl der Testdatenpunkte
n_test = len(test)

# Platzhalter für Vorhersagen
predictions = pd.DataFrame(index=test.index)

# Erstellen des Index mit dem gewünschten Datumbereich für die Vorhersage
forecast_index = pd.period_range(start='2024-06', end='2025-06', freq='M')

# Erstellen des leeren DataFrames mit dem Index
forecasts = pd.DataFrame(index=forecast_index)

# die Funktionen werden nun ausgeführt und die Dataframes mit den berechneten Werten gefüllt
arith_predictions = arithmetisches_mittel(train, test)
naive_predictions = naive_methode(train, test)
seasonal_naive_predictions = saisonale_naive(train, test)
exp_smooth_predictions = exponentielle_glättung(train, test)

# Reset index für Predictions DataFrame
predictions_reset = predictions.reset_index()

# Schmelzen des DataFrames für Plotly Express
predictions_melted = predictions_reset.melt(id_vars=['Jahr', 'Monat'], 
                                            value_vars=['Arithmetisches Mittel', 'Naive Methode', 'Saisonale naive', 'Exponentielle Glättung'],
                                            var_name='Methode', 
                                            value_name='land_PM10')

# das Datum wird in das richtige Format gebracht, für das bessere darstellen beider Dataframes
predictions_melted['Datum'] = pd.to_datetime(predictions_melted['Jahr'].astype(str) + '-' + predictions_melted['Monat'].astype(str))

# Liniengrafik erstellen
fig = px.line(predictions_melted, x='Datum', y='land_PM10', color='Methode',
              labels={'land_PM10': 'land_PM10', 'Methode': 'Methode', 'Datum': 'Datum'},
              title='Vergleich der Vorhersagemethoden für Werte PM10 auf dem Land')

# Das Datum wird nun auch in das richtige Format gebracht
test.index = test.index.map(lambda x: f'{x[0]}-{x[1]:02d}')

# Hinzufügen der Testdaten
fig.add_scatter(x=test.index, y=test.values, mode='lines', name='Testdaten (2023)')

# nun wird auch der Index des Forecasts zu dem gleichen Format
forecasts.index = forecasts.index.to_timestamp()
# Linien für den Forecast erstellen
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Arithmetisches Mittel'], mode='lines', name='Arithmetisches Mittel Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Naive Mathode'], mode='lines', name='Naive Methode Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Saisonale naive'], mode='lines', name='Saisonale naive Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Exponentielle Glättung'], mode='lines', name='Exponentielle Glättung Forecast'))


# Grafik anzeigen
fig.show()

nun werden die Mean-Absolute-Error Fehler der einzelnen Methoden für die Testdaten und Predictions angegeben

In [42]:
mae_arith = mean_absolute_error(test, arith_predictions)
print(mae_arith)
mae_naive = mean_absolute_error(test, naive_predictions)
print(mae_naive)
mae_seasonal_naive = mean_absolute_error(test, seasonal_naive_predictions)
print(mae_seasonal_naive)
mae_exp_smooth = mean_absolute_error(test, exp_smooth_predictions)
print(mae_exp_smooth)

2.7267600483593513
2.4048124616343265
4.210487171408856
1.9711652371751789


In [43]:
# Erstellen von Spalten für Jahr und Monat
selected_df3['Jahr'] = selected_df3.index.year
selected_df3['Monat'] = selected_df3.index.month

# Gruppierung nach Jahr und Monat, Berechnung des Durchschnitts
monthly_mean = selected_df3.groupby(['Jahr', 'Monat']).mean()

# Aufteilung in Trainings- und Testdaten
train = monthly_mean[monthly_mean.index < (2023, 6)]['stadt_PM2_5']
test = monthly_mean[monthly_mean.index >= (2023, 6)]['stadt_PM2_5']

# Anzahl der Testdatenpunkte
n_test = len(test)

# Platzhalter für Vorhersagen
predictions = pd.DataFrame(index=test.index)

# Erstellen des Index mit dem gewünschten Datumbereich für die Vorhersage
forecast_index = pd.period_range(start='2024-06', end='2025-06', freq='M')

# Erstellen des leeren DataFrames mit dem Index
forecasts = pd.DataFrame(index=forecast_index)

# die Funktionen werden nun ausgeführt und die Dataframes mit den berechneten Werten gefüllt
arith_predictions = arithmetisches_mittel(train, test)
naive_predictions = naive_methode(train, test)
seasonal_naive_predictions = saisonale_naive(train, test)
exp_smooth_predictions = exponentielle_glättung(train, test)

# Reset index für Predictions DataFrame
predictions_reset = predictions.reset_index()

# Schmelzen des DataFrames für Plotly Express
predictions_melted = predictions_reset.melt(id_vars=['Jahr', 'Monat'], 
                                            value_vars=['Arithmetisches Mittel', 'Naive Methode', 'Saisonale naive', 'Exponentielle Glättung'],
                                            var_name='Methode', 
                                            value_name='stadt_PM2_5')

# das Datum wird in das richtige Format gebracht, für das bessere darstellen beider Dataframes
predictions_melted['Datum'] = pd.to_datetime(predictions_melted['Jahr'].astype(str) + '-' + predictions_melted['Monat'].astype(str))

# Liniengrafik erstellen
fig = px.line(predictions_melted, x='Datum', y='stadt_PM2_5', color='Methode',
              labels={'stadt_PM2_5': 'stadt_PM2_5', 'Methode': 'Methode', 'Datum': 'Datum'},
              title='Vergleich der Vorhersagemethoden für Werte PM2.5 in der Stadt')

# Das Datum wird nun auch in das richtige Format gebracht
test.index = test.index.map(lambda x: f'{x[0]}-{x[1]:02d}')

# Hinzufügen der Testdaten
fig.add_scatter(x=test.index, y=test.values, mode='lines', name='Testdaten (2023)')

# nun wird auch der Index des Forecasts zu dem gleichen Format
forecasts.index = forecasts.index.to_timestamp()
# Linien für den Forecast erstellen
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Arithmetisches Mittel'], mode='lines', name='Arithmetisches Mittel Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Naive Mathode'], mode='lines', name='Naive Methode Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Saisonale naive'], mode='lines', name='Saisonale naive Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Exponentielle Glättung'], mode='lines', name='Exponentielle Glättung Forecast'))


# Grafik anzeigen
fig.show()

nun werden die Mean-Absolute-Error Fehler der einzelnen Methoden für die Testdaten und Predictions angegeben

In [44]:
mae_arith = mean_absolute_error(test, arith_predictions)
print(mae_arith)
mae_naive = mean_absolute_error(test, naive_predictions)
print(mae_naive)
mae_seasonal_naive = mean_absolute_error(test, seasonal_naive_predictions)
print(mae_seasonal_naive)
mae_exp_smooth = mean_absolute_error(test, exp_smooth_predictions)
print(mae_exp_smooth)

2.5712883632015955
2.425620733850368
4.163644337907787
3.2606743766099413


In [45]:
# Erstellen von Spalten für Jahr und Monat
selected_df4['Jahr'] = selected_df4.index.year
selected_df4['Monat'] = selected_df4.index.month

# Gruppierung nach Jahr und Monat, Berechnung des Durchschnitts
monthly_mean = selected_df4.groupby(['Jahr', 'Monat']).mean()

# Aufteilung in Trainings- und Testdaten
train = monthly_mean[monthly_mean.index < (2023, 6)]['stadt_PM10']
test = monthly_mean[monthly_mean.index >= (2023, 6)]['stadt_PM10']

# Anzahl der Testdatenpunkte
n_test = len(test)

# Platzhalter für Vorhersagen
predictions = pd.DataFrame(index=test.index)

# Erstellen des Index mit dem gewünschten Datumbereich für die Vorhersage
forecast_index = pd.period_range(start='2024-06', end='2025-06', freq='M')

# Erstellen des leeren DataFrames mit dem Index
forecasts = pd.DataFrame(index=forecast_index)

# die Funktionen werden nun ausgeführt und die Dataframes mit den berechneten Werten gefüllt
arith_predictions = arithmetisches_mittel(train, test)
naive_predictions = naive_methode(train, test)
seasonal_naive_predictions = saisonale_naive(train, test)
exp_smooth_predictions = exponentielle_glättung(train, test)

# Reset index für Predictions DataFrame
predictions_reset = predictions.reset_index()

# Schmelzen des DataFrames für Plotly Express
predictions_melted = predictions_reset.melt(id_vars=['Jahr', 'Monat'], 
                                            value_vars=['Arithmetisches Mittel', 'Naive Methode', 'Saisonale naive', 'Exponentielle Glättung'],
                                            var_name='Methode', 
                                            value_name='stadt_PM10')

# das Datum wird in das richtige Format gebracht, für das bessere darstellen beider Dataframes
predictions_melted['Datum'] = pd.to_datetime(predictions_melted['Jahr'].astype(str) + '-' + predictions_melted['Monat'].astype(str))

# Liniengrafik erstellen
fig = px.line(predictions_melted, x='Datum', y='stadt_PM10', color='Methode',
              labels={'stadt_PM10': 'stadt_PM10', 'Methode': 'Methode', 'Datum': 'Datum'},
              title='Vergleich der Vorhersagemethoden für Werte PM10 in der Stadt')

# Das Datum wird nun auch in das richtige Format gebracht
test.index = test.index.map(lambda x: f'{x[0]}-{x[1]:02d}')

# Hinzufügen der Testdaten
fig.add_scatter(x=test.index, y=test.values, mode='lines', name='Testdaten (2023)')

# nun wird auch der Index des Forecasts zu dem gleichen Format
forecasts.index = forecasts.index.to_timestamp()
# Linien für den Forecast erstellen
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Arithmetisches Mittel'], mode='lines', name='Arithmetisches Mittel Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Naive Mathode'], mode='lines', name='Naive Methode Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Saisonale naive'], mode='lines', name='Saisonale naive Forecast'))
fig.add_trace(go.Scatter(x=forecasts.index, y=forecasts['Exponentielle Glättung'], mode='lines', name='Exponentielle Glättung Forecast'))


# Grafik anzeigen
fig.show()

nun werden die Mean-Absolute-Error Fehler der einzelnen Methoden für die Testdaten und Predictions angegeben

In [46]:
mae_arith = mean_absolute_error(test, arith_predictions)
print(mae_arith)
mae_naive = mean_absolute_error(test, naive_predictions)
print(mae_naive)
mae_seasonal_naive = mean_absolute_error(test, seasonal_naive_predictions)
print(mae_seasonal_naive)
mae_exp_smooth = mean_absolute_error(test, exp_smooth_predictions)
print(mae_exp_smooth)

4.879344947455475
5.133567124624068
8.30461566253362
5.7510318353816485
